<a href="https://colab.research.google.com/github/NikoDhrmwn/PadE/blob/main/PadE_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [17]:
train_dir="/content/drive/MyDrive/rice-leaf-desease/training"
val_dir="/content/drive/MyDrive/rice-leaf-desease/validation"
Labels=[]

for file in os.listdir(train_dir):
    Labels+=[file]

Label=[]

for i in range(len(Labels)):
    Label+=[i]
    
mapping = dict(zip(Labels, Label)) 
reverse_mapping = dict(zip(Label, Labels)) 

def mapper(value):
    return reverse_mapping[value]

dataset=[]
valset=[]
count=0
for file in os.listdir(train_dir):
    path=os.path.join(train_dir,file)
    t=0
    for im in os.listdir(path):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(32,32))
        image=img_to_array(image)
        image=image/255.0
        dataset+=[[image,count]]
        t+=1
    count=count+1
count=0
for file in os.listdir(val_dir):
    path=os.path.join(val_dir,file)
    t=0
    for im in os.listdir(path):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(32,32))
        image=img_to_array(image)
        image=image/255.0
        valset+=[[image,count]]
        t+=1
    count=count+1

In [18]:
data,labels0=zip(*dataset)
test,testlabels0=zip(*valset)
labels1=to_categorical(labels0)
labels=np.array(labels1)
data=np.array(data)
test=np.array(test)
trainx,testx,trainy,testy=train_test_split(data,labels,test_size=0.2,random_state=44)

print(trainx.shape)
print(testx.shape)
print(trainy.shape)
print(testy.shape)

(12640, 32, 32, 3)
(3160, 32, 32, 3)
(12640, 4)
(3160, 4)


In [19]:
datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=50,zoom_range=0.8,
                        width_shift_range=0.25,height_shift_range=0.4,shear_range=0.4,fill_mode="nearest")

In [22]:
model = tf.keras.models.Sequential([ 
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation='softmax')
  ])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 30, 30, 16)        448       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 15, 15, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 13, 13, 32)        4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 6, 6, 32)         0         
 g2D)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 1152)              0         
                                                                 
 dense_8 (Dense)             (None, 128)              

In [28]:
history = model.fit(datagen.flow(trainx,trainy,batch_size=75),validation_data=(testx,testy),epochs=15)

Epoch 1/15
169/169 [==============================] - 17s 99ms/step - loss: 0.3185 - accuracy: 0.8725 - val_loss: 0.2193 - val_accuracy: 0.9111
Epoch 2/15
169/169 [==============================] - 17s 99ms/step - loss: 0.3148 - accuracy: 0.8716 - val_loss: 0.1882 - val_accuracy: 0.9278
Epoch 3/15
169/169 [==============================] - 17s 102ms/step - loss: 0.2742 - accuracy: 0.8882 - val_loss: 0.1664 - val_accuracy: 0.9383
Epoch 4/15
169/169 [==============================] - 17s 99ms/step - loss: 0.2695 - accuracy: 0.8931 - val_loss: 0.1474 - val_accuracy: 0.9481
Epoch 5/15
169/169 [==============================] - 17s 99ms/step - loss: 0.2475 - accuracy: 0.9005 - val_loss: 0.1540 - val_accuracy: 0.9316
Epoch 6/15
169/169 [==============================] - 17s 103ms/step - loss: 0.2442 - accuracy: 0.9013 - val_loss: 0.2204 - val_accuracy: 0.9082
Epoch 7/15
169/169 [==============================] - 17s 99ms/step - loss: 0.2374 - accuracy: 0.9025 - val_loss: 0.1589 - val_accurac

In [32]:
image=load_img("/content/drive/MyDrive/rice-leaf-desease/validation/Brown spot/Brown_spot_original_DSC_0100.jpg_6f9b7dc7-db21-46ae-be8c-4e082e5d7868.jpg", target_size=(32,32))

image=img_to_array(image) 
image=image/255.0
prediction_image=np.array(image)
prediction_image= np.expand_dims(image, axis=0)
prediction=model.predict(prediction_image)
value=np.argmax(prediction)
move_name=mapper(value)
print("Prediction is {}.".format(move_name))

Prediction is Brown spot.
